# Studi Kasus Pertemuan ke-13: Dimensionality Reduction
Dalam studi kasus ini, anda akan menggunakan _Principal Component Analysis_ (PCA) pada dataset MNIST.     
Tujuan penggunaan PCA adalah untuk mengurangi jumlah fitur dari dataset MNIST.         
Berikut adalah catatan mengenai _Principal Component Analysis_ dari [AI Flash Cards](https://aiflashcards.com/).

<center>
<img src="Principal_Component_Analysis.png" width=700/>
</center>
   
Dua pertanyaan yang muncul adalah   
1. Apakah PCA selalu dapat meningkatkan kecepatan dari _training_ model?
2. Bagaimana akurasi atau _performance_ model pada dataset yang sudah dikurangi fiturnya? Masih OK? Atau lebih jelek? Atau malahan meningkat?

Untuk menjawab dua pertanyaan tersebut, anda akan melakukan dua eksperimen, yaitu:
1. Membandingkan kecepatan dari _training_ model _random forest_ pada dataset MNIST sebelum dan sesudah dikenakan proses PCA.
2. Membandingkan _performance_ _random forest_ pada dataset MNIST sebelum dan sesudah dikenakan proses PCA.

**2372061 - Laura**

Kita import-import library yang dibutuhkan.

In [1]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [2]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

Mari kita load dataset MNIST.

In [4]:
mnist = fetch_openml('mnist_784', as_frame=False)

Mari kita lihat _shape_ dari `mnist.data` dengan menggunakan _attribute_ `shape`.

In [5]:
##############################
# Write Your Code Here
##############################
mnist.data.shape

(70000, 784)

`mnist.data` terdiri dari 70.000 (tujuh puluh ribu) _instances_.    
Mari kita bagi dataset menjadi _train set_ sebanyak 60.000 (enam puluh ribu) dan _test set_ sebanyak 10.000 (sepuluh ribu).


In [6]:
##############################
# Write Your Code Here
##############################
# Ambillah 60.000 elemen dari mnist
X_train = mnist.data[:60000]
y_train = mnist.target[:60000]

##############################
# Write Your Code Here
##############################
# Ambillah sisanya (10.000 elemen) dari mnist
X_test = mnist.data[60000:]
y_test = mnist.target[60000:]

Mari kita buat `RandomForestClassifier` dengan `n_estimators=100` dan `random_state=42`.

In [7]:
##############################
# Write Your Code Here
##############################
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

Kita hitung juga waktu training yang dibutuhkan.     
Perhatikan **wall time**.

In [8]:
%time rnd_clf.fit(X_train, y_train)

CPU times: user 57.4 s, sys: 124 ms, total: 57.5 s
Wall time: 58.7 s


RandomForestClassifier(random_state=42)

Selanjutnya, kita gunakan model yang sudah dilatih untuk memprediksi test set.    
**Hint**: Gunakan method `predict`.

In [9]:
##############################
# Write Your Code Here
##############################
y_pred = rnd_clf.predict(X_test)

Hitunglah skor akurasinya `accuracy_score` dari prediksi pada `y_test`.

In [10]:
##############################
# Write Your Code Here
##############################
accuracy_score(y_test, y_pred)

0.9705

## _Dimensionality Reduction_ pada MNIST dan Random Forest

Selanjutnya, kita gunakan PCA to reduce the dataset's dimensionality dengan _explained variance ratio_ sebesar 95%.

In [11]:
from sklearn.decomposition import PCA

##############################
# Write Your Code Here
##############################
# 1. Buat dulu instance-nya
# 2. Panggil method fit_transform dari instance untuk mengubah X_train
pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

Selanjutnya, kita latih _Random Forest classifier_ pada dataset yang sudah dikurangi dimensinya.      
Kita hitung juga berapa lama proses training tersebut.   
   
**Catatan**: Waktu yang dibutuhkan dicatat oleh **Wall time**-nya   .

In [15]:
rnd_clf_with_pca = RandomForestClassifier(n_estimators=100, random_state=42)
%time rnd_clf_with_pca.fit(X_train_reduced, y_train)

CPU times: user 2min 44s, sys: 238 ms, total: 2min 44s
Wall time: 2min 45s


RandomForestClassifier(random_state=42)

### Pertanyaan refleksi
- Bagaimana perbandingan waktu _training_ _random forest_ pada dataset lengkap dan waktu _training_ _random forest_ pada dataset yang sudah direduksi?

**Jawab:**
Pada kasus ini, Random Forest saat data masih lengkap memiliki waktu training lebih cepat (58.7 s) dibandingkan setelah direduksi (2min 45s)


Selanjutnya, mari kita evaluasi _random forest_ pada _test set_

Mari kita transform `X_test` dengan `pca`.

In [16]:
##############################
# Write Your Code Here
##############################
X_test_reduced = pca.transform(X_test)

Selanjutnya, kita prediksi `X_test_reduced`.

In [17]:
##############################
# Write Your Code Here
##############################
y_pred = rnd_clf_with_pca.predict(X_test_reduced)

Akhirnya, kita hitung akurasi test set-nya

In [18]:
##############################
# Write Your Code Here
##############################
accuracy_score(y_test, y_pred)


0.9488

### Pertanyaan refleksi
- Bagaimana perbandingan akurasi _random forest_ pada dataset lengkap dan akurasi pada dataset yang sudah direduksi?
- Ketika PCA digunakan, apakah akurasi _random forest_ **menurun jauh** dibandingkan random forest ketika PCA tidak digunakan?   

**Jawab:**
1. Akurasi model Random Forest Classifier pada saat dataset lengkap terbukti lebih besar (97%) daripada akurasi setelah data direduksi (94.8%)
2. Pada saat PCA akurasi Random FOrest menurun jauh, sejauh 3%

## _Dimensionality Reduction_ pada Dataset dan _Stochastic Gradient Descent_ (SGD)

Sekarang kita akan ganti model, dari **random forest** menjadi **stochastic gradient descent** (SGD).

In [19]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
%time sgd_clf.fit(X_train, y_train)

CPU times: user 3min 25s, sys: 583 ms, total: 3min 25s
Wall time: 3min 27s


SGDClassifier(random_state=42)

Selanjutnya, model SGD yang sudah dilatih digunakan untuk memprediksi `X_test`.

In [20]:
##############################
# Write Your Code Here
##############################
y_pred = sgd_clf.predict(X_test)

Hitunglah skor akurasi pada test set-nya.

In [21]:
##############################
# Write Your Code Here
##############################
accuracy_score(y_test, y_pred)

0.874

### Pertanyaan refleksi
- Bagaimanakah perbandingan waktu _training_ `SGDClassifier` dengan waktu _training_ `RandomForestClassifier` pada dataset lengkap?

**Jawab**:     
_Tulis jawaban Anda di sini_

Saatnya, kita coba melatih model SGD tapi kali ini dengan menggunakan dataset yang sudah direduksi.

In [22]:
sgd_clf_with_pca = SGDClassifier(random_state=42)
%time sgd_clf_with_pca.fit(X_train_reduced, y_train)

CPU times: user 47.3 s, sys: 57.9 ms, total: 47.4 s
Wall time: 47.5 s


SGDClassifier(random_state=42)

### Pertanyaan refleksi
- Bagaimana perbandingan waktu _training_ SGD pada dataset lengkap dan waktu _training_ SGD pada dataset yang sudah direduksi?

**Jawab**:     
Pada SGD Classifier, kita bisa lihat bahwa waktu training dataset yang telah direduksi jauh lebih cepat (47.5 s) dibandingkan waktu training data set lengkap (3 mnt 27s)


Mari kita gunakan model SGD untuk memprediksi `X_test_reduced`

In [23]:
##############################
# Write Your Code Here
##############################
y_pred = sgd_clf_with_pca.predict(X_test_reduced)

Hitunglah skor akurasi pada test set ini.

In [24]:
##############################
# Write Your Code Here
##############################
accuracy_score(y_test, y_pred)

0.8959

### Pertanyaan refleksi
- Bagaimana perbandingan akurasi SGD pada dataset lengkap dan akurasi SGD pada dataset yang sudah direduksi?
- Ketika PCA digunakan, apakah akurasi SGD **menurun jauh** dibandingkan ketika PCA tidak digunakan?   

**Jawab**:      
1. Akurasi SGD dengan dataset lengkap adalah 87.4%, sedangkan dengan dataset yang direduksi adalah 89.59%. Hal ini menunjukkan bahwa training dengan dataset reducted lebih cepat.
2. Akurasi SGD menurun lumayan jauh dibandingkan ketika belum menggunakan PCA, perbedaannya hingga 2-3%

### Pertanyaan refleksi
Coba anda pikirkan kembali dua pertanyaan yang ditanyakan di awal, yaitu:  
1. Apakah PCA selalu dapat meningkatkan kecepatan dari pelatihan model?
2. Bagaimana akurasi atau _performance_ model pada dataset yang sudah dikurangi fiturnya? Masih OK? Atau lebih jelek? Atau malahan meningkat?


**Jawab**:      
1. Tidak selalu, tergantung pada Model yang digunakan. Karena Training Time Random FOrest Classifier jauh lebih lama ketika digunakan PCA, sedangkan pada SGD CLassifier jauh lebih cepat setelah menggunakan PCA

2. Tergantung pada model, karena pada Random Forest Classifier bisa dilihat bahwa akurasi setelah PCA malah lebih menurun dibanding data lengkap, sedangkan akurasi pada SDG Classifier meningkat tinggi setelah digunakan PCA.

<center>
    <h1>The End</h1>
</center>    